# Using `tables_io` to write to an hdf5 file from multiple places

In [ ]:
# Standard imports
import os
from collections import OrderedDict
import numpy as np
import tables_io
from tables_io.testUtils import make_test_data

### Some test data.

Ok, lets make some test data write it to an hdf5 file.

In [ ]:
data = make_test_data()

### Some utility functions to get the output data shape

In [ ]:
def get_shapes_and_type(aDict):
    return OrderedDict([(key, (val.shape, val.dtype)) for key, val in aDict.items()])

def modify_shape(shapeIn, length):
    shapeList = list(shapeIn)
    shapeList[0] = length
    return tuple(shapeList)

def set_lengths(aDict, length):
    return OrderedDict([(key, (modify_shape(val[0], length), val[1])) for key, val in aDict.items()])        

### Here we get the output shapes and data types

In [ ]:
dout = set_lengths(get_shapes_and_type(data['data']), 10000)

### Here we allocate the file

In [ ]:
os.unlink('test_multi_write.hdf5')
group, fout = tables_io.io.initializeHdf5Write('test_multi_write.hdf5', 'data', **dout)

### Here we fill the file

In [ ]:
for i in range(10):
    data = make_test_data()
    start = i*1000
    end = (i+1)*1000
    tables_io.io.writeDictToHdf5Chunk(group, data['data'], start, end)

### Here write the metadata and close the file

In [ ]:
tables_io.io.finalizeHdf5Write(fout, 'md', **data['md'])

### Here we read the file in chunks

In [ ]:
for chunks in tables_io.io.iterHdf5ToDict('test_multi_write.hdf5', 1000, 'data'):
    print("%i:%i %s %s %s" % (chunks[0], chunks[1], 
                              chunks[2]['matrix'].shape, chunks[2]['vect'].shape, chunks[2]['scalar'].shape))